## K-NEAREST NEIGHBORS
Từ một tập dữ liệu cho trước, khi có một dữ liệu mới, mô hình sẽ chọn ra K điểm dữ liệu gần nhất, sau đó xem xét nhãn của K điểm dữ liệu này và quyết định xem điểm mới này thuộc lớp nào?

In [60]:
import numpy as np
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split # for splitting data
from sklearn.metrics import accuracy_score

class Model:
    def __init__(self, n_neighbors=1, p=2):
        self.data   = None
        self.target = None
        self.n_neighbors = n_neighbors
        self.p = p
    
    def fit(self, data, target):
        self.data   = data
        self.target = target
        self.data2  = np.sum(data*data, axis=1)

    def dist_ps_fast(self, data):
        if self.p == 1:
            return np.sum(np.abs(self.data - data))
        if self.p == 2:
            return self.data2 - 2 * self.data @ data
    
    def predict(self, dataset):
        result = []
        for data in dataset:
            distances   = self.dist_ps_fast(data)        
            indices     = np.argsort(distances)  
            indices     = indices[:self.n_neighbors]
            y_neighbors = [self.target[idx] for idx in indices]
            result.append(round(np.mean(y_neighbors)))
        return np.array(result)

In [61]:
np.random.seed(7)
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=130)

#### Sử dụng Model triển khai bên trên

In [62]:
model = Model(n_neighbors=3, p=2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy of 1NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 1NN: 93.85 %


#### Sử dụng Scikit-Learn

In [63]:
model = neighbors.KNeighborsClassifier(n_neighbors=3, p=2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy of 1NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 1NN: 93.85 %


### Trọng số

Với thuật toán trước đó K hàng xóm gần điểm sẽ tiến hành biểu quyết, giá trị của các lá phiếu là công bằng như sau.  

Tuy nhiên, chúng ta cũng có thể nghĩ rằng, những hàng xóm gần điểm dữ liệu hơn sẽ đáng tin cậy hơn, và ta có thể cho rằng lá phiếu của những người gần nhất sẽ đáng tin cậy nhất

#### Trọng số dựa trên khoảng cách

In [64]:
model = neighbors.KNeighborsClassifier(n_neighbors = 7, p = 2, weights='distance')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 7NN (1/distance weights): {100*accuracy_score(y_test, y_pred):.2f} %")

Accuracy of 7NN (1/distance weights): 94.62 %


#### Trọng số tự định nghĩa

Một các đánh trọng số khác thường được sử dụng

$$
w_i = \exp\left( \frac{-\|\mathbf{z} - \mathbf{x}_i \|_2^2}{\sigma^2} \right)
$$

In [65]:
def myweight(distances, sigma = 0.7):
    return np.exp(-distances**2/sigma**2)

model = neighbors.KNeighborsClassifier(n_neighbors = 7, p = 2, weights = myweight)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 7NN (customized weights): {100*accuracy_score(y_test, y_pred):.2f} %")

Accuracy of 7NN (customized weights): 94.62 %
